In [1]:
import pandas as pd 
import numpy as np 
import matplotlib.pyplot as plt 

In [2]:
min_lat = 36
min_lon = 338
max_lat = 71
max_lon = 41

In [3]:
station_locations = pd.read_csv('supermag-stations-info.csv')

In [4]:
stations_to_remove = ['A09', 'A10', 'A11', 'ALT', 'ARK', 'ASA', 'ASH', 'B01', 
						'BJI', 'C09', 'CGO', 'CPL', 'CPY', 'CWE', 'DRB', 'E01', 
						'E02', 'E03', 'E04', 'EUA', 'FTN', 'FVE', 'GLK', 'GRK', 
						'KAU', 'KGD', 'KOR', 'KZN', 'LNN', 'M02', 'M11', 'MCE', 
						'MHV', 'MNK', 'MSK', 'MZH', 'NAD', 'NKK', 'NR2', 'NVL', 
						'PKS', 'PNL', 'POD', 'PRG', 'R01', 'R02', 'R03', 'R04', 
						'R05', 'R06', 'R07', 'R08', 'R09', 'R10', 'RSV', 'SAH', 
						'SAS', 'SKD', 'SMA', 'SUT', 'T26', 'T27', 'T60', 'T62', 
						'TKT', 'TLK', 'TOL', 'TOR', 'TTB', 'TUL', 'W01', 'W02', 
						'W03', 'W04', 'W05', 'WSE', 'WTK', 'YSS', 'KHS', 'BEY', 'KLI']
station_locations = station_locations[station_locations['IAGA'].isin(stations_to_remove) == False]

In [5]:
df = station_locations[(station_locations['GEOLAT'] >= min_lat) & (station_locations['GEOLAT'] <= max_lat) & \
	((station_locations['GEOLON'] >= min_lon) | (station_locations['GEOLON'] <= max_lon))]

df.reset_index(inplace=True,drop=True)

In [6]:
stations = df['IAGA'].tolist()

In [7]:
station_dict = {}
for station in stations:
	temp_df = pd.read_feather(f'../data/supermag/{station}.feather')
	temp_df = temp_df[['Date_UTC','dbht','MLT']]
	temp_df.dropna(subset=['dbht'], inplace=True)
	temp_df.set_index('Date_UTC', drop=True, inplace=True)
	station_dict[station] = {}
	station_dict[station]['station_df'] = temp_df

In [8]:
station_dict['MEK']['station_df']

,dbht,MLT
Date_UTC,,
2004-10-01 00:01:00,0.141421,2.57
2004-10-01 00:02:00,0.316228,2.58
2004-10-01 00:03:00,0.223607,2.60
2004-10-01 00:04:00,0.300000,2.62
2004-10-01 00:05:00,0.360555,2.63
...,...,...
2019-12-30 23:56:00,0.223607,1.96
2019-12-30 23:57:00,0.141421,1.98
2019-12-30 23:58:00,0.282843,1.99


In [9]:
main = []
for station in station_dict.keys():
	temp_df = station_dict[station]['station_df']
	difference_df = pd.DataFrame()
	for stat in station_dict.keys():
		if (station == stat) or (stat in main):
			continue
		difference_df[stat] = (temp_df['dbht']-station_dict[stat]['station_df']['dbht']).abs()
	
	difference_df.dropna(how='all', inplace=True)
	print(difference_df)
	station_dict[station]['difference_df'] = difference_df
	main.append(station)


                          LRV  LOV       LER       VAL       HAD       ESK  \
Date_UTC                                                                     
2004-10-01 00:01:00  0.041421  NaN  0.041421  0.141421  0.858579  0.141421   
2004-10-01 00:02:00  0.216228  NaN  0.316228  0.316228  0.316228  0.316228   
2004-10-01 00:03:00  1.263000  NaN  0.123607  0.796197  0.781381  0.223607   
2004-10-01 00:04:00  1.186607  NaN  0.300000  0.704988  0.300000  0.300000   
2004-10-01 00:05:00  2.141444  NaN  0.360555  0.644432  0.639445  0.360555   
...                       ...  ...       ...       ...       ...       ...   
2019-12-30 23:56:00       NaN  NaN  0.076393  0.082185  0.082185  0.023607   
2019-12-30 23:57:00       NaN  NaN  0.041421  0.141421  0.082185  0.158579   
2019-12-30 23:58:00       NaN  NaN  0.059236  0.182843  0.059236  0.255674   
2019-12-30 23:59:00       NaN  NaN  0.000000  0.000000  0.223607  0.223607   
2019-12-31 00:00:00       NaN  NaN  0.100000  0.182843  0.123607

: 

: 